# Лабораторная работа 7. Управление ресурсами в однопроцессорной системе с неоднородными заявками
Выполнил: Пакало Александр Сергеевич, студент РТ5-81Б

## Задание 1
В однопроцессорную систему случайным образом поступают на выполнение $m$
разных типов программ, отличающихся известной трудоемкостью
$Q_1, Q_2,\dots, Q_m$.
Входящий поток простейший с интенсивностью $\lambda$.

Представив данную систему как одноканальную СМО с неограниченной очередью,
вычислить среднее время обслуживания программ, считая длительность
обслуживания случайной величиной (теоретически и экспериментально).

Реализовать алгоритм SPT, выбирая из очереди заявки в соответствии с их
приоритетом по трудоемкости. Рассчитать среднее время обслуживания программ.
Сравнить полученные результаты.

Реализовать алгоритм RR при заданном кванте времени $q$. Оценить среднее время
обслуживания программ. Сравнить полученные результаты. Выяснить, как влияет
величина кванта на среднее время обслуживания программ.

In [1]:
Variant <- 5
set.seed(Variant)
m <- sample(c(6:20), 1)
lambda <- runif(1, 0.1, 2)
Q <- rexp(m, 0.3)
q <- sample(c(1:4), 1)
View(data.frame(m, q, lambda))
print(Q)

m,q,lambda
<int>,<int>,<dbl>
7,1,1.401915


[1] 0.2417931 1.3403831 0.1863999 2.0529013 0.2552007 2.9172096 6.6709868


Заведем таблицу результатов

In [2]:
results <- data.frame("-", "-", "-", "-")
colnames(results) <- c("M/M/1/infty theoretical", "M/M/1/infty practical", "SPT", "Round Robin")
results

M/M/1/infty theoretical,M/M/1/infty practical,SPT,Round Robin
<chr>,<chr>,<chr>,<chr>
-,-,-,-


### СМО вида $М/М/1/\infty$
Представим данную систему как одноканальную СМО с неограниченной очередью.

#### Теоретически

In [3]:
t2 <- mean(Q)
mu <- 1 / t2
mu

[1] 0.5122623

In [4]:
y <- lambda / mu
y

[1] 2.736714

Так как $y > 1$, поменяем $\lambda$.

In [5]:
lambda <- 0.3

In [6]:
t2 <- mean(Q)
mu <- 1 / t2
mu

[1] 0.5122623

In [7]:
y <- lambda / mu
y

[1] 0.5856375

In [8]:
results[1] <- 1 / mu / (1 - y)
results

M/M/1/infty theoretical,M/M/1/infty practical,SPT,Round Robin
<dbl>,<chr>,<chr>,<chr>
4.711152,-,-,-


#### Численно

In [ ]:
if (!require("simmer")) {
    install.packages("simmer")
}
library(simmer)

env <- simmer("SuperDuperSim")
env

In [ ]:
programs <- trajectory("programs' path") %>%
    seize("server", amount = 1) %>%
    timeout(function() rexp(1, mu)) %>%
    release("server", amount = 1)

In [ ]:
SIMULATION_TIME <- 10000

env %>%
    add_resource("server", 1) %>%
    add_generator("programs", programs, function() rexp(1, lambda)) %>%
    run(until = SIMULATION_TIME)

In [ ]:
arrivals <- env %>%
    get_mon_arrivals()
arrivals

In [ ]:
results[2] <- mean(arrivals %>% with(end_time - start_time))
results

### Алгоритм SPT

In [ ]:
if (!require("simmer")) {
    install.packages("simmer")
}
library(simmer)

spt.env <- simmer("SuperDuperSptSim")
spt.env

Добавим $m$ генераторов. Каждый будет иметь приоритет в зависимости от
скорости выполнения программы. Генератор, создающий программы с наибольшей
длительностью выполнения, будет иметь наименьший приоритет.

In [ ]:
SIMULATION_TIME <- 100000

spt.env %>%
    add_resource("server", 1)

Q_sorted_decr <- sort(Q, decreasing = TRUE)

spt.programs_trajectory <- function(time_to_execute) {
    return(
        trajectory("programs' path") %>%
            seize("server", 1) %>%
            timeout(time_to_execute) %>%
            release("server", 1)
    )
}

for (Q_i in seq_along(Q_sorted_decr)) {
    priority <- Q_i

    name <- paste0("programs", Q_i)

    spt.env %>% add_generator(
        name,
        spt.programs_trajectory(Q_sorted_decr[Q_i]),
        priority = priority,
        preemptible = priority,
        distribution = function() rexp(1, lambda / m)
    )
}

spt.env %>%
    run(until = SIMULATION_TIME)

In [ ]:
spt.arrivals <- spt.env %>%
    get_mon_arrivals()
spt.arrivals

In [ ]:
results[3] <- mean(spt.arrivals %>% with(end_time - start_time))
results

### Алгоритм Round Robin
Реализуем Round Robin с помощью simmer.

In [9]:
if (!require("simmer")) {
    install.packages("simmer")
}
library(simmer)

rr.env <- simmer("SuperDuperRoundRobinSim")
rr.env

Loading required package: simmer



simmer environment: SuperDuperRoundRobinSim | now: 0 | next: 
{ Monitor: in memory }

In [10]:
rr.execute_program_for_quant <- trajectory() %>%
    seize("server", 1) %>%
    timeout(function() min(get_attribute(rr.env, "execution_time"), q)) %>%
    set_attribute("execution_time", -q, mod = "+") %>%
    release("server", 1) %>%
    leave(
        function() get_attribute(rr.env, "execution_time") > 0
    )

rr.programs <- trajectory() %>%
    set_attribute("execution_time", function() sample(Q, 1)) %>%
    handle_unfinished(
        trajectory() %>%
            log_(function() paste0("preempteed with execution_time: ", get_attribute(rr.env, "execution_time"))) %>%
            join(rr.execute_program_for_quant)
    ) %>%
    join(rr.execute_program_for_quant)

In [11]:
SIMULATION_TIME <- 10000

rr.env %>%
    add_resource("server", 1, preemptive = TRUE) %>%
    add_generator(
        "programs",
        rr.programs,
        priority = 1,
        preemptible = 1,
        restart = TRUE,
        distribution = function() rexp(1, lambda)
    ) %>%
    run(until = SIMULATION_TIME)

11.7477: programs4: preempteed with execution_time: 1.91720958101713
12.7477: programs4: preempteed with execution_time: 0.917209581017126
14.9771: programs5: preempteed with execution_time: 1.91720958101713
16.2189: programs5: preempteed with execution_time: 0.917209581017126
22.4522: programs7: preempteed with execution_time: 1.05290133909633
23.6386: programs7: preempteed with execution_time: 0.0529013390963278
29.5807: programs9: preempteed with execution_time: 1.91720958101713
30.7671: programs9: preempteed with execution_time: 0.917209581017126
45.1908: programs12: preempteed with execution_time: 1.05290133909633
46.1908: programs12: preempteed with execution_time: 0.0529013390963278
48.2265: programs13: preempteed with execution_time: 1.05290133909633
49.2265: programs13: preempteed with execution_time: 0.0529013390963278
70.8768: programs17: preempteed with execution_time: 1.91720958101713
71.8768: programs17: preempteed with execution_time: 0.917209581017126
76.1852: programs2

simmer environment: SuperDuperRoundRobinSim | now: 10000 | next: 10000.2102771205
{ Monitor: in memory }
{ Resource: server | monitored: TRUE | server status: 1(1) | queue status: 1(Inf) }
{ Source: programs | monitored: 1 | n_generated: 2992 }

In [12]:
rr.arrivals <- rr.env %>%
    get_mon_arrivals()
rr.arrivals

name,start_time,end_time,activity_time,finished,replication
<chr>,<dbl>,<dbl>,<dbl>,<lgl>,<int>
programs0,1.282143,1.537344,0.2552007,TRUE,1
programs1,2.753483,2.995276,0.2417931,TRUE,1
programs2,7.774030,7.960430,0.1863999,TRUE,1
programs3,8.101920,8.288320,0.1863999,TRUE,1
programs4,10.747733,13.664943,2.9172096,TRUE,1
programs6,14.200234,15.218854,0.2417931,TRUE,1
programs5,13.977061,17.136064,2.9172096,TRUE,1
programs8,22.042705,22.638626,0.1863999,TRUE,1
programs7,21.452226,23.691528,2.0529013,TRUE,1


In [13]:
results[4] <- mean(rr.arrivals %>% with(end_time - start_time))
results

M/M/1/infty theoretical,M/M/1/infty practical,SPT,Round Robin
<dbl>,<chr>,<chr>,<dbl>
4.711152,-,-,4.440699


Как видно, практические вычисления совпадают теоретическими с некоторой
погрешностью, которая уменьшается при увеличении числа экспериментов.

При этом система, выполненная с помощью алгоритма Round Robin оказалась
быстрее обычной системы, а система, реализованная с алгоритмом SPT - самой
быстрой.